# Mettalex Autonomous Market Maker
Date: 2020-12-17

Author: Matt McDonnell, Fetch.ai/Mettalex

This notebook describes calcuations used to design an improved version of the Mettalex Autonomous Market Maker (AMM) to provide fair liquidity-sensitive pricing.

Currently the Mettalex system is running on the Ethereum blockchain which has the benefit of allowing reuse of existing DeFi protocols at the expense of some limitations in the allowed efficient mathematical operations. The Mettalex system roadmap plans to transition the autonomous market making functionality to the Fetch.ai network to make use of more advanced optimization techniques. As such the functionality described here should be regarded as an intial version of the AMM that will be extended as more network capabilities become available.

The initial version of the Mettalex AMM described in [amm_balancer.ipynb](amm_balancer.ipynb) makes use of a private Balancer Smart Pool internally to provide the functionality of swapping between coin token (USDT intially) and long or short position tokens. The internal Balancer AMM is controlled to set prices of long and short tokens to reflect market demand and constraints on the token prices.

The new version of the Mettalex AMM discussed in this notebook builds on the analysis of the previous notebook to propose changes to the Balancer invariant to achieve system constraints.

In [2]:
# Initial notebook setup: functions are stored in ./calc/amm_math.py 
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'calc'))

from amm_math import (
    calc_spot_price, calc_token_balance, calc_out_given_in, calc_in_given_out, 
    set_amm_state, set_amm_state_rebalance, get_amm_spot_prices, get_amm_balance, 
    simple_swap_from_coin, simple_swap_to_coin, simulate_swaps_from_coin,
    calc_balancer_invariant, mint_redeem, deposit_withdraw, perform_action,
    plot_action, plot_action_orderbook, print_state_change, perform_action_sequence
)

import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [3]:
# Weights
w_c, w_l, w_s = sp.symbols('w_c w_l w_s', positive=True)
# Token balances
x_c, x_l, x_s = sp.symbols('x_c x_l x_s', positive=True)
# Prices - in units of locked up collateral
v = sp.symbols('v', positive=True) 
# Swap fee in fraction of input tokens
s_f = sp.symbols('s_f')
# Decimal scaling 
d_c, d_p = sp.symbols('d_c d_p', positive=True)
# Collateral backing L + S 
# e.g. for $100.000_000_000_000_000_000 = 1.000_000 mt
# this is 100*10**18 / 1*10**6 = 10**14
C = sp.symbols('C', positive=True)

# Input token amounts for swaps
a_c, a_l, a_s = sp.symbols('a_c a_l a_s', positive=True)
x_m = sp.symbols('x_m')  # Amount of coin tokens used for minting long and short

In [4]:
help(calc_spot_price)

Help on function calc_spot_price in module amm_math:

calc_spot_price(bI, wI, bO, wO, sF=0)
    calcSpotPrice                                                                             //
     sP = spotPrice                                                                            //
     bI = tokenBalanceIn                ( bI / wI )         1                                  //
     bO = tokenBalanceOut         sP =  -----------  *  ----------                             //
     wI = tokenWeightIn                 ( bO / wO )     ( 1 - sF )                             //
     wO = tokenWeightOut                                                                       //
     sF = swapFee



In [5]:
help(calc_balancer_invariant)

Help on function calc_balancer_invariant in module amm_math:

calc_balancer_invariant(x_c, x_l, x_s, w_c, w_l, w_s)
    calcBalancerInvariant - return invariant satisfied by Balancer pool for swaps
    Uses discrete inputs rather than state vector for ease of vectorized plotting use.
    To use state vector: calc_balancer_invariant(*state)



From the [Balancer whitepaper](https://balancer.finance/whitepaper/):

> The bedrock of Balancer’s exchange functions is a surface defined by constraining a value function $V$
 — a function of the pool’s weights and balances — to a constant. We will prove that this surface implies a spot price at each point such that, no matter what exchanges are carried out, the share of value of each token in the pool remains constant.

In [6]:
V_0 = calc_balancer_invariant(x_c, x_l, x_s, w_c, w_l, w_s)

In [7]:
V_0

x_c**w_c*x_l**w_l*x_s**w_s

The [Balancer whitepaper](https://balancer.finance/whitepaper/) shows that the vaulue function is related to the token spot prices by the ratio of partial derivative.  This is shown below by comparing the partial derivative ratio with the explict expression for the spot price.

In [8]:
# Spot price is given by tangent to invariant
V_0.diff(x_l)/V_0.diff(x_c)

w_l*x_c/(w_c*x_l)

In [9]:
calc_spot_price(x_c, w_c, x_l, w_l)

w_l*x_c/(w_c*x_l)

# Modifying the Invariant
Investigate simple modifications to the Balancer invariant such as adding a function, multiplying a function or scaling the inputs by a function, such that we preserve the sum $1L + 1S = \$C$.

(a day later) some further thought about how the constraints work leads to the feeling that this approach may need modification, see section 'Starting from the Constraints' below.

In [22]:
def sum_of_spots(V):
    return V.diff(x_l)/V.diff(x_c) + V.diff(x_s)/V.diff(x_c)

In [23]:
sum_of_spots(V_0)

w_l*x_c/(w_c*x_l) + w_s*x_c/(w_c*x_s)

In [27]:
f = sp.Function('f')

## Add function
Try adding a function that only depends on the coin balance first. Solving the sum of position tokens invariant condition requires solving a differential equation.

In [39]:
f_a = sp.dsolve(sum_of_spots(V_0 + f(x_c)).simplify() - C, f(x_c), ics={f(0): 0})

In [41]:
f_a

Eq(f(x_c), x_c**w_c*x_l**(w_l - 1)*x_s**(w_s - 1)*(-C*w_c*x_l*x_s - C*x_l*x_s + w_l*x_c*x_s + w_s*x_c*x_l)/(C*(w_c + 1)))

In [47]:
V_0 + f_a.rhs

x_c**w_c*x_l**w_l*x_s**w_s + x_c**w_c*x_l**(w_l - 1)*x_s**(w_s - 1)*(-C*w_c*x_l*x_s - C*x_l*x_s + w_l*x_c*x_s + w_s*x_c*x_l)/(C*(w_c + 1))

In [49]:
(V_0 + f_a.rhs).subs({x_c: x_c + a_c, x_l: x_l - a_l})

x_s**w_s*(a_c + x_c)**w_c*(-a_l + x_l)**w_l + x_s**(w_s - 1)*(a_c + x_c)**w_c*(-a_l + x_l)**(w_l - 1)*(-C*w_c*x_s*(-a_l + x_l) - C*x_s*(-a_l + x_l) + w_l*x_s*(a_c + x_c) + w_s*(a_c + x_c)*(-a_l + x_l))/(C*(w_c + 1))

In [51]:
((V_0 + f_a.rhs).subs({x_c: x_c + a_c, x_l: x_l - a_l}) - (V_0 + f_a.rhs)).simplify()

(C*x_s**w_s*(w_c + 1)*(-x_c**w_c*x_l**w_l + (a_c + x_c)**w_c*(-a_l + x_l)**w_l) + x_c**w_c*x_l**(w_l - 1)*x_s**(w_s - 1)*(C*w_c*x_l*x_s + C*x_l*x_s - w_l*x_c*x_s - w_s*x_c*x_l) + x_s**(w_s - 1)*(a_c + x_c)**w_c*(-a_l + x_l)**(w_l - 1)*(C*w_c*x_s*(a_l - x_l) + C*x_s*(a_l - x_l) + w_l*x_s*(a_c + x_c) - w_s*(a_c + x_c)*(a_l - x_l)))/(C*(w_c + 1))

In [79]:
# Solver fails
# sp.solve(
#     (V_0 + f_a.rhs).subs({x_c: x_c + a_c, x_l: x_l - a_l}) - (V_0 + f_a.rhs),
#     a_l
# )

In [53]:
f_a.rhs.expand()

-C*w_c*x_c**w_c*x_l**w_l*x_s**w_s/(C*w_c + C) - C*x_c**w_c*x_l**w_l*x_s**w_s/(C*w_c + C) + w_l*x_c*x_c**w_c*x_l**w_l*x_s**w_s/(C*w_c*x_l + C*x_l) + w_s*x_c*x_c**w_c*x_l**w_l*x_s**w_s/(C*w_c*x_s + C*x_s)

In [56]:
sp.solve(
    V_0.subs({x_c: x_c + a_c, x_l: x_l - a_l}) - V_0,
    a_l
)[0].simplify()

-x_c**(w_c/w_l)*x_l*(a_c + x_c)**(-w_c/w_l) + x_l

## Multiply function

In [40]:
f_m = sp.dsolve(sum_of_spots(V_0 * f(x_c)).simplify() - C, f(x_c))

In [42]:
f_m

Eq(f(x_c), C1*x_c**(-w_c)*exp(x_c*(w_l/x_l + w_s/x_s)/C))

## Scale coin balance

In [58]:
V_1 = calc_balancer_invariant(f(x_c), x_l, x_s, w_c, w_l, w_s)

In [61]:
f_c = sp.dsolve(sum_of_spots(V_1).simplify() - C, f(x_c), ics={f(0): 1})

In [62]:
f_c

Eq(f(x_c), exp(x_c*(w_l*x_s + w_s*x_l)/(C*w_c*x_l*x_s)))

In [73]:
sp.log(f_c.rhs).expand()

w_l*x_c/(C*w_c*x_l) + w_s*x_c/(C*w_c*x_s)

In [78]:
# Solver fails
# sp.solve(
#     V_1.subs({f(x_c): f_c.rhs}).subs({x_c: x_c + a_c, x_l: x_l - a_l}) - V_1.subs({f(x_c): f_c.rhs}),
#     a_l
# )[0].simplify()

In [75]:
V_1.subs({f(x_c): f_c.rhs})

x_l**w_l*x_s**w_s*exp(x_c*(w_l*x_s + w_s*x_l)/(C*x_l*x_s))

# Starting from the Constraints
The idea of constant level sets of a value function creating constraints on system state (including prices) is discussed in ['From Curved Bonding to Configuration Spaces'](https://epub.wu.ac.at/7385/) by Zargham, Shorish, and Paruch (2020).  

The previous section started to investigate ways to alter the value function to introduce new constraints. However, some further thought leads to the conclusion that the existing Balancer value function implicit state constraint 'the share of value of each token in the pool remains constant' may conflict with the desired new constraint 'the sum of position token spot prices equals the underlying collateral'.

In this section we look at starting from a set of constraints and see if it is possible to derive the corresponding value function.  We can then use this value function to determine allowed state changes e.g. for a swap the number of output tokens for an initial state and given number of input tokens.

This approach feels familiar to the Lagrangian dynamics approach in classical physics (the author's background).  In economics the standard approach seems to be start from a value function (a.k.a. utility function) and derive substitution functions that give prices.  Here we attempt to solve the inverse problem.

As a starting point, we consider deriving the Balancer value function (a [Cobb-Douglas Utility Function](https://www.wikiwand.com/en/Cobb%E2%80%93Douglas_production_function)) from the set of constraints for swaps 'the share of value of each token in the pool remains constant'.  

The state of the system can be defined by three token balances $x_c$, $x_l$, $x_s$, and three weights $w_c$, $w_l$, $w_s$, giving 6 variables in total.

The share of value constraints on token swaps gives 3 constraints, and we can choose to introduce a further token weight sum to 1 constraint.  This leaves 2 free parameters and by inspection we see that the value function we hope to derive is invariant to scaling of the function (and possibly all parameters).  While this proves nothing it is at least reassuring that we might be on the right track.

In [11]:
V = sp.Function('V')(x_c, x_l, x_s, w_c, w_l, w_s)

In [15]:
sp.Equality(V.diff(x_c)/V.diff(x_l), -x_l/x_c*w_c/w_l)  # TODO: check signs and ratio direction

Eq(Derivative(V(x_c, x_l, x_s, w_c, w_l, w_s), x_c)/Derivative(V(x_c, x_l, x_s, w_c, w_l, w_s), x_l), -w_c*x_l/(w_l*x_c))

To do: read [SymPy PDE Solver doc](https://docs.sympy.org/latest/modules/solvers/pde.html) and author to refresh their memory on PDEs.  Multiplicative separable function $X_c(x_c)*X_l(x_l)*X_s(x_s)$ is actual solution with $X_i(x_i) = x_i^{w_i}$ so see if we can find that solution with the SymPy tools.